<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorial/02_intermediate/Convolutinal_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()
tfe = tf.contrib.eager
tfk = tf.keras

In [10]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 1024
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tfk.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [19]:
x_train_ = tf.transpose(tf.convert_to_tensor(x_train, dtype=tf.float32), 
                        [0, 3, 1, 2])
y_train_ = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))

print(x_train_.shape)
print(y_train_.shape)

(50000, 3, 32, 32)
(50000, 10)


In [20]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

print(train_dataset)

<MapDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [21]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
print(test_dataset)

<MapDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [0]:
class Cifar10Model(tfk.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tfk.Sequential([
            tfk.layers.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])

        self.conv_block2 = tfk.Sequential([
            tfk.layers.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])
        
        self.conv_block3 = tfk.Sequential([
            tfk.layers.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])
        
        self.flatten = tfk.layers.Flatten()
        self.fc1 = tfk.layers.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_initializer=tf.initializers.variance_scaling,
            kernel_regularizer=tfk.regularizers.l2(l=0.001))
        self.dropout = tfk.layers.Dropout(0.8)
        self.fc2 = tfk.layers.Dense(10)
        self.softmax = tfk.layers.Softmax()

    def call(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [0]:
model = Cifar10Model()

def loss_fn(model, x, y):
    y_pre = model(x)
    return tfk.losses.categorical_crossentropy(y, y_pre)

def accuracy(model, x, y):
    y_pre = model(x)
    return tfk.metrics.categorical_accuracy(y, y_pre)

optimizer = tf.train.AdamOptimizer(learning_rate)

In [45]:
for j in range(num_epochs):
    
    running_loss = 0
    running_acc = 0

    for i, (x_, y_) in enumerate(train_dataset):
        
        with tf.device("/gpu:0"):
            with tf.GradientTape() as tape:
                loss = loss_fn(model, x_, y_)
            acc = accuracy(model, x_, y_)
            grads = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(zip(grads, model.variables))
            running_loss += tf.reduce_mean(loss)
            running_acc += tf.reduce_mean(acc)
    
    print("-----epoch {} -----".format(j + 1))
    print("loss: ", running_loss.numpy()/(i + 1))
    print("acc: ", running_acc.numpy()/(i + 1))    

-----epoch 1 -----
loss:  1.9387904575892858
acc:  0.29365350762192083
-----epoch 2 -----
loss:  1.5621210215043049
acc:  0.43229566301618305
-----epoch 3 -----
loss:  1.4413411665935905
acc:  0.48328862871442524
-----epoch 4 -----
loss:  1.3570086420798788
acc:  0.5132680231211136
-----epoch 5 -----
loss:  1.300194720832669
acc:  0.5373571746203364
-----epoch 6 -----
loss:  1.2398374907824459
acc:  0.5596067856769172
-----epoch 7 -----
loss:  1.197290459457709
acc:  0.5755784949477838
-----epoch 8 -----
loss:  1.154668379803093
acc:  0.5920901785091478
-----epoch 9 -----
loss:  1.130118311667929
acc:  0.600405790367905
-----epoch 10 -----
loss:  1.1001540981993383
acc:  0.6116759631098533


In [47]:
test_accuracy = 0
for i, (x_, y_) in enumerate(test_dataset):
    test_accuracy += tf.reduce_mean(accuracy(model, x_, y_))
test_accuracy /= i + 1

print("test accuracy {:0.3f}".format(test_accuracy.numpy()))

test accuracy 0.605
